In [0]:
%run "./Process or Create Table"

In [0]:
process_or_create_table("user_table", None, summary=True, delete=False)

building user_table from existing table files
+------------+
|record_count|
+------------+
|     1987897|
+------------+

+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+----+----------------------+----+-----+-------------+------------+------+----------------------+-------------------+------------+
|average_stars|compliment_cool|compliment_cute|compliment_funny|compliment_hot|compliment_list|compliment_more|compliment_note|compliment_photos|compliment_plain|compliment_profile|compliment_writer|cool|                 elite|fans|funny|         name|review_count|useful|               user_id|      yelping_since|friend_count|
+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+----+----------------

In [0]:
spark.sql("""
SELECT user_id, name, elite
FROM user_table
""").show(truncate=False)

+----------------------+-------------+-------------------------+
|user_id               |name         |elite                    |
+----------------------+-------------+-------------------------+
|Rx4Jl0SK0Vu5xlw3RPbdmA|Avery        |                         |
|qL2n1KDDQccYD-Lx4MWmVw|Tanya        |                         |
|P5_E3H8dWyTr4lJiaNUfTw|Abindas      |                         |
|EF8JYiN8ohkAA8cipaRUpA|Zia-Kashanti |                         |
|lxpavrqhbfOvxIepv41RdQ|Rick         |                         |
|P_2DQxHwueC4Jm2qM-4Uuw|Margie       |2017,2018,2019,20,20,2021|
|3dEhqPG8dMNQkSpz472JMA|Tyler        |                         |
|5qK2shRKiP9PGCKUPWbEaQ|Doug         |                         |
|_rS4QzfgKkMhwBDk9K2GvA|Arnie Beauyet|                         |
|XKNe7pMsiHzdM6f8HUHFlw|Ryan         |                         |
|1QVt2Y1u6aZU94l_f3s-cQ|Ro           |                         |
|ScdVF7L_oT_X7AZNGeg-6Q|Shannon      |                         |
|1NVrBnkXEKrvWK7yNpqyEg|D

In [0]:
df_clean_users = spark.sql("""
SELECT user_id, name, yelping_since, review_count, friend_count, REGEXP_REPLACE(elite, '20,20', '2020') AS elite
FROM user_table
""")
print(f"record count: {df_clean_users.count()}")
df_clean_users.show(truncate=False)
df_clean_users.createOrReplaceTempView("clean_users")

record count: 1987897
+----------------------+-------------+-------------------+------------+------------+------------------------+
|user_id               |name         |yelping_since      |review_count|friend_count|elite                   |
+----------------------+-------------+-------------------+------------+------------+------------------------+
|Rx4Jl0SK0Vu5xlw3RPbdmA|Avery        |2016-11-16 20:58:16|5           |142         |                        |
|qL2n1KDDQccYD-Lx4MWmVw|Tanya        |2009-01-09 00:14:29|42          |123         |                        |
|P5_E3H8dWyTr4lJiaNUfTw|Abindas      |2015-08-17 00:10:07|7           |251         |                        |
|EF8JYiN8ohkAA8cipaRUpA|Zia-Kashanti |2010-05-27 12:23:58|7           |46          |                        |
|lxpavrqhbfOvxIepv41RdQ|Rick         |2011-09-30 19:36:53|42          |6           |                        |
|P_2DQxHwueC4Jm2qM-4Uuw|Margie       |2017-06-02 00:17:41|334         |296         |2017,2018,2019

In [0]:
df_users = spark.sql("""
WITH elite_array AS 
     (SELECT user_id, SPLIT(elite,"\\\s*,\\\s*") AS elite_years
      FROM clean_users),
      
     exploded_elite AS
     (SELECT user_id, EXPLODE(elite_years) AS elite_years
     FROM elite_array),
     
     converted_elite AS
     (SELECT user_id, CAST(elite_years AS Integer)
     FROM exploded_elite),
     
     collected_elite AS
     (SELECT user_id, COLLECT_LIST(elite_years) AS elite_years
     FROM converted_elite
     GROUP BY user_id)
    
      
SELECT U.user_id, U.name, U.yelping_since, 
       U.review_count AS reviews_written, U.friend_count, 
       ARRAY_SORT(E.elite_years) AS elite_years,
       (SIZE(elite_years) <> 0) AS is_elite,
       ARRAY_MIN(E.elite_years) AS first_elite
FROM clean_users AS U INNER JOIN collected_elite AS E
ON U.user_id = E.user_id
""")
print(f"record count: {df_users.count()}")
df_users.printSchema()
df_users.show(100, truncate=False)
df_users.createOrReplaceTempView("users")

record count: 1987897
root
 |-- user_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- yelping_since: string (nullable = true)
 |-- reviews_written: long (nullable = true)
 |-- friend_count: integer (nullable = true)
 |-- elite_years: array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- is_elite: boolean (nullable = false)
 |-- first_elite: integer (nullable = true)

+----------------------+-----------+-------------------+---------------+------------+------------------------------------------------------------------------------+--------+-----------+
|user_id               |name       |yelping_since      |reviews_written|friend_count|elite_years                                                                   |is_elite|first_elite|
+----------------------+-----------+-------------------+---------------+------------+------------------------------------------------------------------------------+--------+-----------+
|--1orhUoGFSdHXsoxqQc8g|K

In [0]:
%run "./Process or Create Table"

In [0]:
process_or_create_table("ssa_gender_table", None, summary=True, delete=False)

building ssa_gender_table from existing table files
+------------+
|record_count|
+------------+
|      101338|
+------------+

+----------+-----+----+-----+------+------------------+
|      name|women| men|total|gender|      gender_ratio|
+----------+-----+----+-----+------+------------------+
|    Skylor|  418|1052| 1470|     M|0.7156462585034014|
|   Burnell|  593|2785| 3378|     M|0.8244523386619301|
|   Almedia|  530|   0|  530|     F|               1.0|
|Marcelline| 1065|   0| 1065|     F|               1.0|
|       Bud|    0|9449| 9449|     M|               1.0|
|   Rosetta|27985|  44|28029|     F|0.9984301972956581|
|     Eliya|  686| 145|  831|     F| 0.825511432009627|
|  Khennedi|   24|   0|   24|     F|               1.0|
|    Dedric|   20|1843| 1863|     M|0.9892646269457863|
|    Fallon|10066| 532|10598|     F|0.9498018494055482|
+----------+-----+----+-----+------+------------------+



In [0]:
df_user_gender = spark.sql("""
SELECT U.*, 
       COALESCE(gender, 'Unknown') AS gender,
       COALESCE(gender_ratio, 1.0) AS gender_ratio
FROM users AS U LEFT OUTER JOIN ssa_gender_table AS G
ON LOWER(U.name) = LOWER(G.name)
 """)
print(f"Record count: {df_user_gender.count()}")
df_user_gender.show(truncate=False)
df_user_gender.createOrReplaceTempView("user_gender")

Record count: 1987897
+----------------------+-------+-------------------+---------------+------------+------------------+--------+-----------+-------+------------------+
|user_id               |name   |yelping_since      |reviews_written|friend_count|elite_years       |is_elite|first_elite|gender |gender_ratio      |
+----------------------+-------+-------------------+---------------+------------+------------------+--------+-----------+-------+------------------+
|--1orhUoGFSdHXsoxqQc8g|Keny   |2011-03-18 05:56:39|41             |70          |[]                |false   |null       |M      |0.9765258215962441|
|--3Hl2oAvTPlq-f7KtogJg|Rogan  |2013-09-11 04:19:10|11             |1           |[]                |false   |null       |M      |0.9450072358900145|
|--4AjktZiHowEIBCMd4CZA|Bert   |2015-06-01 16:58:12|57             |1           |[]                |false   |null       |M      |0.9807917202666355|
|--5EJ-TYQgbeViSGYRIYXQ|Eric   |2014-10-09 03:31:34|1              |1           |[] 

In [0]:
process_or_create_table("user_gender_table", "df_user_gender", summary=True, delete=True)

Saving the df_user_gender DataFrame as table: user_gender_table
+------------+
|record_count|
+------------+
|     1987897|
+------------+

+----------------------+---------+-------------------+---------------+------------+-----------+--------+-----------+------+------------------+
|               user_id|     name|      yelping_since|reviews_written|friend_count|elite_years|is_elite|first_elite|gender|      gender_ratio|
+----------------------+---------+-------------------+---------------+------------+-----------+--------+-----------+------+------------------+
|---2PmXbF47D870stH1jqA|    Susan|2012-10-24 13:20:46|             36|         420|         []|   false|       null|     F|0.9977396781152311|
|--1vcleKg-_ORilNx_RaDw| Michelle|2018-11-25 19:52:17|              3|          56|         []|   false|       null|     F| 0.996020560843788|
|--1xed48O2w6o-TLqxScdQ|Elizabeth|2013-08-12 22:56:43|              3|           1|         []|   false|       null|     F|0.9968809772017858|
|-

In [0]:
df_user_gender_viz = spark.sql("""
SELECT gender, 
       COUNT(user_id) AS user_count, 
       ROUND(COUNT(first_elite)/COUNT(gender),4) AS elite_percentage, 
       SUM(reviews_written) AS reviews_written
FROM user_gender_table
GROUP BY gender
ORDER BY user_count DESC
""")
df_user_gender_viz.show()

+-------+----------+----------------+---------------+
| gender|user_count|elite_percentage|reviews_written|
+-------+----------+----------------+---------------+
|      F|   1033782|          0.0538|       24191050|
|      M|    767910|          0.0398|       17785122|
|Unknown|    186205|           0.027|        4529504|
+-------+----------+----------------+---------------+



In [0]:
display(df_user_gender_viz)

gender,user_count,elite_percentage,reviews_written
F,1033782,0.0538,24191050
M,767910,0.0398,17785122
Unknown,186205,0.027,4529504


Databricks visualization. Run in Databricks to view.